<br>

# Molecular Dynamics and Ab Initio Simulations : Developing a Predictive Model for Molecular Energy Configurations

## Table of contents:

* [First view on data](#first-view-on-data)
    * [MO2S4 dataset](#mo2s4-dataset)
    * [Zundel dataset](#zundel-dataset)
* [Data Preprocessing](#data-preprocessing)


This project involves utilizing results from ab initio molecular dynamics simulations to develop a model capable of predicting the energy associated with specific molecular configurations. It leverages two datasets: the first encompasses simulations on the dynamics of the Zundel ion (H2O-H-H2O), while the second focuses on a Mo2S4 aggregate. Each dataset contains about 10,000 atomic configurations and their corresponding potential energies, providing a comprehensive basis for accurate energy prediction in molecular systems.

<br>

<br>

## First view on data <a class="anchor" id="first-view-on-data"></a>

### MO2S4 dataset <a class="anchor" id="mo2s4-dataset"></a>

First, let's try to render a configuration on a 3D plot. We will first look into the MO2S4 dataset by plotting the first and last configurations.

<br>

In [22]:
%matplotlib notebook

In [91]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

MO2S4_file_path = 'data/MO2S4/TRAJEC_short.xyz'

# Re-reading the file to confirm its structure
with open(MO2S4_file_path, 'r') as file:
    MO2S4_file_content = file.readlines()

# Parsing the first configuration from the file
# Skipping the first two lines (number of atoms and comment) and taking the next six lines for the first configuration

def plot_configuration_MO2S4(first_line, last_line):

    configuration = MO2S4_file_content[first_line:last_line]

    # Extracting atom types and their positions
    atom_types = []
    positions = []

    for line in configuration:
        parts = line.split()
        atom_types.append(parts[0])  # Atom type (Mo or S)
        positions.append([float(parts[1]), float(parts[2]), float(parts[3])])  # x, y, z coordinates

    positions = np.array(positions)

    # Creating a 3D scatter plot using 'o' markers to represent atoms
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection='3d')

    # Plotting each atom with 'o' marker
    for i, atom in enumerate(atom_types):
        x, y, z = positions[i]
        color = 'gray' if atom == 'Mo' else 'yellow'  # Gray for Mo, Yellow for S
        ax.scatter(x, y, z, c=color, marker='o', s=100)  # Adjust size as needed

    # Adding labels and title
    ax.set_xlabel('X axis')
    ax.set_ylabel('Y axis')
    ax.set_zlabel('Z axis')
    ax.set_title('3D Visualization of Atomic Configuration with Scatter Plot')

    # Add legend
    ax.scatter([], [], [], c='gray', marker='o', s=100, label='Mo')
    ax.scatter([], [], [], c='yellow', marker='o', s=100, label='S')
    ax.legend()

    # Add a title

    plt.title('Configuration n°' + str(last_line//8))

    plt.show()

plot_configuration_MO2S4(2,8)
plot_configuration_MO2S4(88002,88008)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<br>

We can check these configurations with an image from Jmol:

<br>

#### First configuration :

![image](img/first_config_MO2S4.png)
<br>

#### Last configuration :

![image](img/last_config_MO2S4.png)

<br>

We can see the plotting are correct (even tho there is a slight difference in the position of the atoms, but it is due to the fact that the Jmol image is not in the exact same orientation as the plotting).

<br>


We notice that the configurations are quite different.

Properties of ``TRAJEC_short.xyz`` :
- 11,001 configurations (88,008 lines/8 lines per configuration).
- No units for positions, but it can be assumed that they are in Angstroms ($\AA = 10^-{10}m$).

It is quickly noticed that there are 11,001 lines in the ``energies.out`` file, and the same in the ``potential-energy`` file. It seems that the values in ``potential-energy.txt`` correspond to those in ``energies.out``. We can then verify this:

<br>

In [24]:
MO2S4_energies_out = pd.read_csv("data/MO2S4/energies.out", delim_whitespace=True)
MO2S4_energies_out.head()

,step,none,temp,potential_energy,none.1,none.2,none.3,none.4
0,1,0.00000000,1278.492,-176.957259,-176.893668,0.0,0.000062,1.31
1,5,0.00000000,8620.471,-176.992785,-175.702624,0.0,0.036543,1.56
2,10,0.00000000,954.638,-176.996302,-175.624659,0.0,0.095147,0.84
3,15,0.00000000,319.990,-176.997355,-175.621132,0.0,0.121177,1.05
4,20,0.00000000,273.078,-177.004502,-175.619745,0.0,0.137373,1.32


In [25]:
MO2S4_potential_energy = pd.read_csv('data/MO2S4/potential-energy.txt', delim_whitespace=True, header=None)
MO2S4_potential_energy.columns = ['potential_energy']
MO2S4_potential_energy.head()

,potential_energy
0,-176.957259
1,-176.992785
2,-176.996302
3,-176.997355
4,-177.004502


In [26]:
# On teste si les deux colonnes sont égales
print(energies_out['potential_energy'].equals(potential_energy['potential_energy']))

True


<br>


The two columns quickly lead us to assume that the ``energies.out`` file represents the energies of the 11,001 configurations in the ``TRAJEC_short.xyz`` file. We can then simply use the energies.out file to train our model.

<br>

### Zundel dataset <a class="anchor" id="zundel-dataset"></a>

<br>

This dataset is composed of only two files, ``new_energies_sparse.out`` and ``new_positions_sparse.xyz``. We can directly see that these two files are, resp., the energy table we saw on ``energies.out`` and the ``TRAJEC_short.xyz``. Let's import them and look the same way as the MO2S4.

<br>


In [27]:
zundel_file_path = 'data/zundel_ions/new_positions_sparse.xyz'

# Re-reading the file to confirm its structure
with open(zundel_file_path, 'r') as file:
    zundel_file_content = file.readlines()

# Parsing the first configuration from the file
# Skipping the first two lines (number of atoms and comment) and taking the next six lines for the first configuration

def plot_configuration_zundel(first_line, last_line):

    configuration = zundel_file_content[first_line:last_line]

    # Extracting atom types and their positions
    atom_types = []
    positions = []

    for line in configuration:
        parts = line.split()
        atom_types.append(parts[0])  # Atom type (O or H)
        positions.append([float(parts[1]), float(parts[2]), float(parts[3])])  # x, y, z coordinates

    positions = np.array(positions)

    # Creating a 3D scatter plot using 'o' markers to represent atoms
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection='3d')

    # Plotting each atom with 'o' marker
    for i, atom in enumerate(atom_types):
        x, y, z = positions[i]
        color = 'red' if atom == 'O' else 'gray'  # Gray for H, Red for O
        ax.scatter(x, y, z, c=color, marker='o', s=100)  # Adjust size as needed

    # Adding labels and title
    ax.set_xlabel('X axis')
    ax.set_ylabel('Y axis')
    ax.set_zlabel('Z axis')
    ax.set_title('3D Visualization of Atomic Configuration with Scatter Plot')

    # Add legend
    ax.scatter([], [], [], c='gray', marker='o', s=100, label='H')
    ax.scatter([], [], [], c='red', marker='o', s=100, label='O')
    ax.legend()

    # Add a title

    plt.title('Configuration n°' + str(last_line//8))

    plt.show()

plot_configuration_zundel(2,9)
plot_configuration_zundel(89993,90000)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<br>

Again, we can check these configurations with an image from Jmol:

<br>

#### First configuration :

![image](img/first_config_zundel.png)
<br>

#### Last configuration :

![image](img/last_config_zundel.png)

<br>

We see that ...

<br>

Now let's check the table of energies:

In [28]:
zundel_potential_energy = pd.read_csv('data/zundel_ions/new_energies_sparse.out', delim_whitespace=True, header=None)
zundel_potential_energy.columns = ['step','none','temp','potential_energy','none.1','none.2','none.3','none.4']
zundel_potential_energy.head()

,step,none,temp,potential_energy,none.1,none.2,none.3,none.4
0,1.0,2.0,0.003692,0.003269,0.000096,0.003174,0.000302,95.439881
1,1.0,102.0,0.004527,0.006531,0.002632,0.003899,0.000371,117.256850
2,1.0,202.0,0.003945,0.007161,0.002978,0.004182,0.000398,125.779360
3,1.0,302.0,0.003904,0.005873,0.002675,0.003198,0.000305,96.178014
4,1.0,402.0,0.004485,0.007464,0.003446,0.004018,0.000383,120.833550


## Data Preprocessing <a class="anchor" id="data-preprocessing"></a>

<br>

The goal here is to describe the data and to prepare it for the model. Indeed, we will try to directly use the data as it is, without any preprocessing and use a naive model, and then we will try to preprocess it to see if it improves the model.

<br>

### Naive and linear regression model <a class="anchor" id="naive-and-linear-regression-model"></a>

<br>

We will first try to use the data as it is, without any preprocessing. By this, I mean we will use the coordinates of the atoms as features and the potential energy as the target. Then, we will use the mean of the potential energy as a naive model to predict the potential energy of a configuration and check the score of this model.

In [67]:
# data : MO2S4_file_content (6 lignes par configuration, séparés par 2 lignes, commence à ligne 2)
# Les 6 lignes sont : type d'atome, x, y, z
# on va d'abord récupérer dans l'ordre les positions des atomes dans chaque configuration et les placer dans une liste à 18 dimensions (6*3 dimensions par atome)

def get_liste_configurations_MO2S4():
    liste_configurations = []
    
    for i in range(2, len(MO2S4_file_content), 8):
        configuration = []
        for j in range(6):
            configuration.append(MO2S4_file_content[i+j].split()[1:])
        liste_configurations.append(configuration)
    
    liste_configurations = np.array(liste_configurations, dtype=float)
    
    return liste_configurations

liste_configurations = get_liste_configurations_MO2S4()

print(f" Nombres de configurations : {len(liste_configurations)}"
      f"\n Dimensions de la première configuration : {liste_configurations[0].shape}"
      f"\n Première configuration : \n {liste_configurations[0]}"
      f"\n Moyenne des positions des configurations : \n {np.mean(liste_configurations, axis=0)}"
      f"\n Ecart-type des positions des configurations : \n {np.std(liste_configurations, axis=0)}"
      f"\n Minimum des positions des configurations : \n {np.min(liste_configurations, axis=0)}"
      f"\n Maximum des positions des configurations : \n {np.max(liste_configurations, axis=0)}")


 Nombres de configurations : 11001
 Dimensions de la première configuration : (6, 3)
 Première configuration : 
 [[ 0.746648  1.869656  0.279349]
 [-1.188737  0.558299 -0.089274]
 [ 1.795806  0.234278 -1.878226]
 [-1.139974  2.551034  1.381834]
 [-0.410638  2.128456 -1.909369]
 [ 1.230796 -2.537868  1.275581]]
 Moyenne des positions des configurations : 
 [[0.70660374 2.24968146 0.40045112]
 [0.5333103  1.04449557 0.45250144]
 [0.41124667 1.94025897 0.01663025]
 [0.32677503 2.32664991 0.11183954]
 [0.22970309 1.21658537 0.13250167]
 [1.31070329 1.08543397 0.79624568]]
 Ecart-type des positions des configurations : 
 [[1.00512999 0.91284739 1.2601428 ]
 [1.15492425 0.70500757 0.9058091 ]
 [0.97871082 1.52298532 1.31281585]
 [0.88091546 1.23522166 0.87012479]
 [1.47769804 0.72970001 1.14937565]
 [0.86425788 1.68849477 1.04346837]]
 Minimum des positions des configurations : 
 [[-1.857595  0.289364 -2.367561]
 [-1.875278 -0.750295 -1.389803]
 [-1.824819 -1.258395 -2.064855]
 [-1.342688 -0

In [64]:
# On peut maintenant récupérer les énergies potentielles dans le fichier potential-energy.txt et les placer dans un array numpy

def get_energies_MO2S4():
    energies = []
    
    for i in range(len(MO2S4_potential_energy)):
        energies.append(MO2S4_potential_energy['potential_energy'][i])
    
    energies = np.array(energies, dtype=float)
    
    return energies

energies = get_energies_MO2S4()

print(f" Nombres d'énergies : {len(energies)}"
      f"\n Première énergie : {energies[0]}"
      f"\n Dernière énergie : {energies[-1]}"
      f"\n Moyenne des énergies : {np.mean(energies)}"
      f"\n Ecart-type des énergies : {np.std(energies)}"
      f"\n Minimum des énergies : {np.min(energies)}, indice : {np.argmin(energies)}"
      f"\n Maximum des énergies : {np.max(energies)}, indice : {np.argmax(energies)}")

 Nombres d'énergies : 11001
 Première énergie : -176.9572587946
 Dernière énergie : -176.2992603329
 Moyenne des énergies : -174.78168354970256
 Ecart-type des énergies : 32.3810590146537
 Minimum des énergies : -177.2277536385, indice : 537
 Maximum des énergies : 633.4860704294, indice : 10868


<br>

#### Naive model

On peut maintenant créer un modèle naïf qui va prédire la moyenne des énergies potentielles en utilisant les positions des atomes comme features et les énergies potentielles comme target. La méthode de score utilisée sera le MSE (Mean Squared Error).

<br>

In [86]:
from sklearn.metrics import mean_squared_error

# On peut maintenant créer un modèle naïf qui va prédire la moyenne des énergies potentielles

X = liste_configurations
y = energies

y_pred = np.full(len(energies), np.mean(energies))

score = mean_squared_error(energies, y_pred)

print(f" MSE du modèle naïf : {score}")
print(f" RMSE du modèle naïf : {np.sqrt(score)}")

 MSE du modèle naïf : 1048.5329829104855
 RMSE du modèle naïf : 32.3810590146537


<br>

On obtient un MSE de 1048, ce qui est un score élevé pour des énergies de l'ordre de $10^2$. On peut donc en déduire que le modèle naïf n'est pas adapté pour ce problème. 
On remarque aussi facilement que le RMSE est égal à l'écart-type des énergies, ce qui est logique puisque le modèle naïf prédit la moyenne des énergies, et que l'écart-type est la mesure de dispersion des énergies par rapport à la moyenne.

<br>

#### Linear regression model

On peut maintenant essayer un modèle un peu plus complexe, en utilisant les positions des atomes comme features et les énergies potentielles comme target. On va utiliser un modèle de régression linéaire, et on va séparer les données en un jeu d'entraînement et un jeu de test. On va ensuite calculer le MSE et le RMSE de ce modèle.

<br>

In [89]:
from sklearn.model_selection import train_test_split

# On sépare les données en un jeu d'entraînement et un jeu de test

print(X.shape)
flattened_X = X.reshape(11001, 18)
print(flattened_X.shape)
print(y.shape)
X_train, X_test, y_train, y_test = train_test_split(flattened_X, y, test_size=0.2, random_state=42)

print(f" Dimensions du jeu d'entraînement : {X_train.shape}"
      f"\n Dimensions du jeu de test : {X_test.shape}"
      f"\n Dimensions du jeu d'entraînement : {y_train.shape}"
      f"\n Dimensions du jeu de test : {y_test.shape}")   


(11001, 6, 3)
(11001, 18)
(11001,)
 Dimensions du jeu d'entraînement : (8800, 18)
 Dimensions du jeu de test : (2201, 18)
 Dimensions du jeu d'entraînement : (8800,)
 Dimensions du jeu de test : (2201,)


In [92]:
from sklearn.linear_model import LinearRegression

# On crée le modèle de régression linéaire

linear_regression = LinearRegression()

# On entraîne le modèle

linear_regression.fit(X_train, y_train)

# On prédit les énergies potentielles du jeu de test

y_pred = linear_regression.predict(X_test)

# On calcule le MSE et le RMSE

score = mean_squared_error(y_test, y_pred)

print(f" MSE du modèle de régression linéaire : {score}")
print(f" RMSE du modèle de régression linéaire : {np.sqrt(score)}")


 MSE du modèle de régression linéaire : 821.390444426886
 RMSE du modèle de régression linéaire : 28.659910056154853


<br>

On remarque que l'on a diminué le MSE, mais pas de beaucoup. On peut donc en déduire que le modèle de régression linéaire n'est pas adapté pour ce problème. La question est alors de savoir si le problème vient du modèle ou des données. On va donc essayer d'améliorer les données pour voir si le modèle s'améliore.

<br>

#### Again with Zundel Ions

On va répéter la même chose pour les Zundel Ions.

<br>

In [ ]:
# on refait pareil

### Describing the data with other representations <a class="anchor" id="describing-the-data-with-other-representations"></a>

<br>



In [108]:
from ase.io import read

structure1 = read("data/MO2S4/TRAJEC_short.xyz")

print(structure1)

from ase.visualize import view

view(structure1)

Atoms(symbols='Mo2S4', pbc=False)


<Popen: returncode: None args: ['C:\\Users\\lucas\\anaconda3\\python.exe', '...>